In [ ]:
import pandas as pd
import datetime

In [53]:
bills = pd.read_csv('../../src/bexio/data/raw/bills.csv')
projects = pd.read_csv('../../src/bexio/data/raw/projects.csv')
proposals = pd.read_csv('../../src/bexio/data/raw/proposals.csv')
payments = pd.read_csv('../../src/bexio/data/raw/payments.csv')

In [80]:
merged_df = pd.merge(bills, payments[['Rechnungs-Nr.', 'Rechnungsdatum', 'Anzahl Tage', 'Datum']],
                     left_on='Nr.', right_on='Rechnungs-Nr.', how='left')

merged_df = merged_df[merged_df['Status'].isin(['Bezahlt', 'Offen'])]
merged_df = merged_df.rename(columns={'Datum_y': 'Zahlungsdatum'})
merged_df = merged_df.rename(columns={'Anzahl Tage': 'Rechnungsdatum - Zahlungsdatum'})
merged_df = pd.merge(merged_df, projects[['Titel', 'Frist']], on=['Titel'], how='left')
merged_df['Zahlungsdatum'] = merged_df['Zahlungsdatum'].fillna(merged_df['Frist_x'])

columns_to_drop = [
    'Kontaktnummer', 'Kontaktreferenz', 'Ansprechpartner', 'Verkäufer',
    'Bankkonto', 'Referenz', 'Zahlungsart', 'Zusatztext', 'Leistungszeitraum', 'ESR-Nummer', 'ESR-Nummer (lang)',
    'QR-Referenz', 'MWST 0.00%', 'Datum Zahlungseingang 1', 'Betrag Zahlungseingang 1', 'Konto Zahlungseingang 1',
    'Datum Zahlungseingang 2', 'Betrag Zahlungseingang 2', 'Konto Zahlungseingang 2', 'Rechnungs-Nr.', 'ID', 'Mahnstufe', 'Projekt', 'Rechnungsdatum', 'Frist_y', 'Datum_x', 'Frist_x', 'MWST 7.70%'
]

merged_df.drop(columns=columns_to_drop, inplace=True)
merged_df['Status'] = merged_df['Status'].replace('Offen', 'Zahlung Offen')
merged_df

,Nr.,Titel,Status,Kontakt,Währung,Betrag Netto,Betrag Brutto,Vorgängerdokument,Rechnungsdatum - Zahlungsdatum,Zahlungsdatum
0,RE-00002,"Gesuche - Dorfstrasse 22A/22E, 5624 Bünzen",Bezahlt,Stalder-Rüttimann GmbH,CHF,560.00,603.1,NaN,30.0,'13.05.2022
1,RE-00003,"GEAK®, EFH, Kt. AG",Bezahlt,Herbert Schacher,CHF,798.50,860.0,Auftrag AU-00003,6.0,'19.04.2022
2,RE-00004,"Wärmepumpe Fördergesuch Zürich, Klimastiftung ...",Bezahlt,AQUASANI GmbH,CHF,278.55,300.0,NaN,29.0,'02.06.2022
3,RE-00005,NaN,Bezahlt,AQUASANI GmbH,CHF,464.25,500.0,NaN,29.0,'02.06.2022
4,RE-00008,"GEAK+, Rankstrasse 4, Nussbaumen, EFH",Bezahlt,Ronald Frei und Nadja Lötscher,CHF,1021.35,1100.0,NaN,9.0,'13.05.2022
...,...,...,...,...,...,...,...,...,...,...
206,RE-00251,"FB Geak+, Schulstr.194, Schwaderloch, EFH",Zahlung Offen,"Departement Bau, Verkehr, Umwelt",CHF,1000.00,1000.0,Auftrag AU-00116,NaN,'19.07.2023
207,RE-00252,"EN - Talstr.10, Veltheim AG, MFH",Zahlung Offen,TOKA AG,CHF,2785.50,3000.0,Auftrag AU-00113,NaN,'19.07.2023
208,RE-00253,"G+ Artherstr.132, Oberwil b. Zug, EFH",Zahlung Offen,Stadt Zug,CHF,2414.10,2600.0,Auftrag AU-00101,NaN,'19.07.2023
209,RE-00254,"GEAK Plus - Schöntal 4, Endingen",Zahlung Offen,Sidler Architekten AG,CHF,1935.45,2200.0,Auftrag AU-00109,NaN,'28.07.2023


In [91]:
#todo: offene Projekte zu merged_df hinzufügen. Aufträge mit Teilzahlung soll der "Betrag Netto" und "Betrag Butto" durch die getätigten Zahlung aus merged_df reduziert werden. Ein Abgleich kann auf dem Attribut "Titel" erfolgen.

#todo: Zahlungsdatum in Datum umwandeln

filtered_projects = projects[projects['Status'] == 'Offen'].copy()
filtered_projects.loc[:, 'Status'] = 'Auftrag Offen'

filtered_projects

,ID,Nr.,Titel,Status,Kontakt,Kontaktnummer,Kontaktreferenz,Projekt,Ansprechpartner,Verkäufer,...,Betrag Brutto,Datum,Frist,Bankkonto,Vorgängerdokument,Referenz,MWST 0.00%,MWST 7.70%,Wiederholung,Teilrechnung
12,15,AU-00015,Planung Heizung + Abluft,Auftrag Offen,Sani Project,'000005,6,"Planung H/L, Rainstrasse 52, Nussbaumen",Imhoff Kevin,NaN,...,7323.60,'04.05.2022,'02.06.2022,PostFinance (CHF),Angebot AN-00026,"Rainstrasse 52, Nussbaumen",'0,523.6000,Nein,Ja
21,26,AU-00026,"Planung H/S - Erni, Guggibadstr.6, Schongau",Auftrag Offen,Stalder-Rüttimann GmbH,'000003,4,"Planung H/S - Erni, Guggibadstr.6, Schongau",Imhoff Kevin,NaN,...,9369.90,'25.05.2022,'23.06.2022,PostFinance (CHF),Angebot AN-00041,"Erni, Guggibadstr.6, Schongau",'0,669.9000,Nein,Ja
76,88,AU-00088,"P - Oberebersol 49, Scheuber",Auftrag Offen,Stalder-Rüttimann GmbH,'000003,4,"P - Oberebersol 49, Scheuber",Imhoff Kevin,NaN,...,7323.60,'10.02.2023,'11.03.2023,PostFinance (CHF),Angebot AN-00108,NaN,'0,523.6000,Nein,Ja
91,104,AU-00104,"Geak+, Buchenweg 9, Würenlos",Auftrag Offen,Eric Moesner,'000255,161,NaN,Imhoff Kevin,NaN,...,2100.00,'08.05.2023,'06.06.2023,PostFinance (CHF),Auftrag AU-00083,"Geak+, Buchenweg 9, Würenlos",'0,221.6342,Nein,Nein
92,105,AU-00105,"FB Geak+, Buchenweg 9, Würenlos",Auftrag Offen,"Departement Bau, Verkehr, Umwelt",'000120,26,NaN,Imhoff Kevin,NaN,...,1000.00,'08.05.2023,'06.06.2023,PostFinance (CHF),NaN,NaN,'0,'0,Nein,Nein
95,108,AU-00108,"Geak+, Willadingweg 30/32, Bern, 2 MFH",Auftrag Offen,AWT Treuhand & Verwaltungen AG,'000257,163,NaN,Imhoff Kevin,NaN,...,4000.00,'11.05.2023,'09.06.2023,PostFinance (CHF),Auftrag AU-00090,"Willadingweg 30/32, Bern, 2 MFH",'0,285.9796,Nein,Nein
99,112,AU-00112,"EN - Oberer Rainweg 1, Frick, 2x EFH",Auftrag Offen,TOKA AG,'000240,146,NaN,Imhoff Kevin,NaN,...,1300.00,'26.05.2023,'24.06.2023,PostFinance (CHF),Angebot AN-00137,"Oberer Rainweg 1, Frick, 2x EFH",'0,92.9434,Nein,Nein
104,117,AU-00117,"P - Oberebersol 49, Parzelle 1675",Auftrag Offen,Stalder-Rüttimann GmbH,'000003,4,NaN,Imhoff Kevin,NaN,...,6569.70,'30.05.2023,'28.06.2023,PostFinance (CHF),Angebot AN-00141,"Oberebersol 49, Parzelle 1675",'0,469.7000,Nein,Nein
109,122,AU-00122,"G+, Sandbuckstrasse 6, Schneisingen, EFH",Auftrag Offen,MiKo Gmbh,'000263,169,NaN,Imhoff Kevin,NaN,...,1600.00,'06.06.2023,'05.07.2023,PostFinance (CHF),Angebot AN-00147,"Sandbuckstrasse 6, Schneisingen, EFH",'0,185.8867,Nein,Nein
110,123,AU-00123,"AP - H/L - Gassenacker 10, Rickenbach, MFH",Auftrag Offen,Zahra Chopany,'000198,104,NaN,Imhoff Kevin,NaN,...,4684.95,'29.06.2023,'28.07.2023,PostFinance (CHF),Angebot AN-00154,"Gassenacker 10, 8545 Rickenbach, Neubau MFH",'0,334.9500,Nein,Nein


In [100]:
#todo: doesnt work. should add the filtered_projects rows to merged_df

import datetime

# Iterate over each row in filtered_projects
for i, fp_row in filtered_projects.iterrows():
    # Copy the values from filtered_projects to merged_df
    merged_df.loc[len(merged_df)] = {
        'Nr.': fp_row['Nr.'],
        'Titel': fp_row['Titel'],
        'Status': fp_row['Status'],
        'Kontakt': fp_row['Kontakt'],
        'Währung': fp_row['Währung'],
        'Betrag Netto': fp_row['Betrag Brutto'],
        'Vorgängerdokument': fp_row['Vorgängerdokument'],
        'Zahlungsdatum': datetime.datetime.strptime(fp_row['Frist'], "'%d.%m.%Y") + datetime.timedelta(days=30),
        'Rechnungsdatum - Zahlungsdatum': 0
    }

# Print the updated merged_df
merged_df


,Ansprechpartner,Bankkonto,Betrag Brutto,Betrag Netto,Datum,Frist,ID,Kontakt,Kontaktnummer,Kontaktreferenz,...,Rechnungsdatum - Zahlungsdatum,Referenz,Status,Teilrechnung,Titel,Verkäufer,Vorgängerdokument,Wiederholung,Währung,Zahlungsdatum
0,NaN,NaN,603.1,603.10,NaN,NaN,NaN,Stalder-Rüttimann GmbH,NaN,NaN,...,0,NaN,Bezahlt,NaN,"Gesuche - Dorfstrasse 22A/22E, 5624 Bünzen",NaN,NaN,NaN,CHF,NaT
1,NaN,NaN,860.0,860.00,NaN,NaN,NaN,Herbert Schacher,NaN,NaN,...,0,NaN,Bezahlt,NaN,"GEAK®, EFH, Kt. AG",NaN,Auftrag AU-00003,NaN,CHF,NaT
2,NaN,NaN,300.0,300.00,NaN,NaN,NaN,AQUASANI GmbH,NaN,NaN,...,0,NaN,Bezahlt,NaN,"Wärmepumpe Fördergesuch Zürich, Klimastiftung ...",NaN,NaN,NaN,CHF,NaT
3,NaN,NaN,500.0,500.00,NaN,NaN,NaN,AQUASANI GmbH,NaN,NaN,...,0,NaN,Bezahlt,NaN,NaN,NaN,NaN,NaN,CHF,NaT
4,NaN,NaN,1100.0,1100.00,NaN,NaN,NaN,Ronald Frei und Nadja Lötscher,NaN,NaN,...,0,NaN,Bezahlt,NaN,"GEAK+, Rankstrasse 4, Nussbaumen, EFH",NaN,NaN,NaN,CHF,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,NaN,NaN,NaN,1300.00,NaN,NaN,NaN,TOKA AG,NaN,NaN,...,0,NaN,Auftrag Offen,NaN,"EN - Oberer Rainweg 1, Frick, 2x EFH",NaN,Angebot AN-00137,NaN,CHF,2023-07-24
361,NaN,NaN,NaN,6569.70,NaN,NaN,NaN,Stalder-Rüttimann GmbH,NaN,NaN,...,0,NaN,Auftrag Offen,NaN,"P - Oberebersol 49, Parzelle 1675",NaN,Angebot AN-00141,NaN,CHF,2023-07-28
362,NaN,NaN,NaN,1600.00,NaN,NaN,NaN,MiKo Gmbh,NaN,NaN,...,0,NaN,Auftrag Offen,NaN,"G+, Sandbuckstrasse 6, Schneisingen, EFH",NaN,Angebot AN-00147,NaN,CHF,2023-08-04
363,NaN,NaN,NaN,4684.95,NaN,NaN,NaN,Zahra Chopany,NaN,NaN,...,0,NaN,Auftrag Offen,NaN,"AP - H/L - Gassenacker 10, Rickenbach, MFH",NaN,Angebot AN-00154,NaN,CHF,2023-08-27


In [ ]:
#todo: add Aufwand approx. per month